# Steg 1 — Installera nödvändiga bibliotek

In [ ]:
!pip install py2neo llama_index llama_index.llms.ollama llama_index.embeddings.ollama

# Steg 2 — Anslut till Neo4j

In [ ]:
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "password")

# Steg 3 — Definiera en enkel chunking-funktion

In [ ]:
def chunk_text(text, max_tokens=300):
    """Väldigt enkel chunking: splittar på meningar."""
    sentences = text.split('. ')
    chunks, current_chunk = [], ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_tokens:
            current_chunk += sentence + '. '
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + '. '
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Steg 4 — Ladda in ett testdokument

In [ ]:
test_document = """
GDPR är en europeisk lag som syftar till att stärka individens rättigheter vid behandling av personuppgifter.
Alla företag måste följa GDPR för att säkerställa integritet och transparens. 
Brott mot GDPR kan leda till höga böter. 
Dataskyddsombud ansvarar för efterlevnad av reglerna.
"""
chunks = chunk_text(test_document)
chunks

# Steg 5 — Generera embeddings

In [ ]:
from llama_index.embeddings.ollama import OllamaEmbedding

embed_model = OllamaEmbedding(model_name="mistral")

embedded_chunks = [(chunk, embed_model.get_text_embedding(chunk)) for chunk in chunks]

# Steg 6 — Lagra chunks i Neo4j

In [ ]:
for chunk_text, embedding in embedded_chunks:
    graph.run("""
    CREATE (c:Chunk {text: $text, embedding: $embedding})
    """, text=chunk_text, embedding=embedding)

In [ ]:
#✅ Nu är din dokumentinformation inmatad i grafen!

# Steg 7 — Sök i grafen via embeddings

In [ ]:
query = "Vad handlar GDPR om?"
query_embedding = embed_model.get_text_embedding(query)

results = graph.run("""
    MATCH (c:Chunk)
    WITH c, gds.similarity.cosine(c.embedding, $query_embedding) AS score
    WHERE score > 0.7
    RETURN c.text AS text, score
    ORDER BY score DESC
    LIMIT 5
""", query_embedding=query_embedding).data()

results

# Steg 8 — Bygg upp en prompt från träffarna

In [ ]:
context = "\n\n".join([r['text'] for r in results])

prompt = f"""Context:
{context}

Answer the question:
{query}
"""
print(prompt)

# Steg 9 — Anropa LLM för att svara på frågan

In [ ]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="mistral")

response = llm.complete(prompt)
print(response.text)